In [2]:
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import cdist
import pickle as pk
import gzip

In [ ]:
rgb = [(x,y,z) for x in range(256) for y in  range(256) for z in range(256)]


In [ ]:
def rgb_to_lab(color):
   
    num = 0
    rgb = [0,0,0]
    for clr in color:
        clr = float(clr)/255
        if clr > 0.04045:
            clr = ((clr + 0.055)/1.055)**2.4
        else:
            clr/=12.92
        rgb[num] = clr*100
        num+=1
    xyz = [0,0,0]
    x  = rgb[0]*0.4124 + rgb[1]*0.3576 +rgb[2]*0.1805
    y  = rgb[0]*0.2126 + rgb[1]*0.7152 +rgb[2]*0.0722
    z  = rgb[0]*0.0193 + rgb[1]*0.1192 +rgb[2]*0.9505
    

    xyz[0] = round(x,4)
    xyz[1] = round(y,4)
    xyz[2] = round(z,4)

    xyz[0] = float(xyz[0])/95.047
    xyz[1] = float(xyz[1])/100.0
    xyz[2] = float(xyz[2])/108.883
    num = 0
    for clr in xyz:
        if clr>0.008856:
            clr = clr**(0.3333333333333333)
        else:
            clr = (7.787*clr) + (16/116)
        xyz[num] = clr
        num+=1
    lab = [0,0,0]
    l = (116*xyz[1])-16
    a = 500*(xyz[0]-xyz[1])
    b = 200*(xyz[1]-xyz[2])
    lab[0] = round(l,4)
    lab[1] = round(a,4)
    lab[2] = round(b,4)

    
    return lab

In [ ]:
lab_colors = [tuple(rgb_to_lab(clr)) for clr in rgb]

In [ ]:
color_map = { 
    'Dark Red':(255,0,0),
    'Light Red':(255,102,102),
    'Dark Green':(0,102,51),
    'Green':(0,255,0),
    'Light Green':(51,255,51),
    'Dark Blue':(0,0,204),
    'Light Blue':(51,153,255),
    'Sky Blue':(153,204,255),
    'Light Yellow':(255,255,204),
    'Dark Yellow':(255,255,0),
    'Golden':(255,215,0),
    'Black':(0,0,0),
    'White':(255,255,255),
    #'Silver':(192,192,192),
    'Orange':(255,128,0),
    'Pink':(255,0,255),
    'Purple':(128,0,128),
    'Grey':(224,224,224),
    'Light Brown':(153,76,0),
    'Chocolate':(102,51,0),
    'Brown':(51,25,0),
    'Dark Brown': (102,0,0),
    'Maroon':(128,0,0),
    'Cyan':(0,255,255),
    'Magenta':(255,0,0),
    'Teal':(0,153,153),
    'Skin':(255,204,153),
    'Cream':(255,229,204),
    'Light Orange':(255,178,102),
    'Parrot Green':(0,153,0),
    'Navy Blue':(0,0,102),
    'Redish Pink':(255,51,153)
}

In [ ]:
image_color_map = {key:tuple(rgb_to_lab(val)) for key, val in color_map.items()}

In [ ]:
lookup = defaultdict(int)
mp = {}
key_colors = list(image_color_map.keys())
val_colors_rgb = list(color_map.values())
val_colors = np.array(list(image_color_map.values()))

In [ ]:
def get_color(color):
    return 'rgb({},{},{})'.format(*color)

In [ ]:
for clr in lab_colors :
    if tuple(clr) in lookup:
        dist = lookup[tuple(clr)]
    else:
        dist = np.argmin(cdist(np.array(val_colors),np.array(clr).reshape(-1,3), metric = 'cityblock'))
        lookup[tuple(clr)] = dist
    mp[clr] = get_color(color_map[key_colors[dist]])

In [ ]:

with gzip.open('map.pkl.gz', 'wb') as f:
    pk.dump(mp, f, protocol=pk.HIGHEST_PROTOCOL)

In [ ]:
rgb2lab_map = {clr:tuple(rgb_to_lab(clr)) for clr in rgb}

In [ ]:
import pickle
import gzip

# Save with gzip compression
with gzip.open('maprgb2lab.pkl.gz', 'wb') as f:
    pickle.dump(rgb2lab_map, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open("maprgb2lab.pkl", "wb") as file:
#     pk.dump(rgb2lab_map, file)

In [ ]:
# import json

# def map_dict(d):
#     return {str(k): v for k, v in d.items()}


# with open('rgb2lab_map.txt', 'w') as file:
#     file.write(json.dumps(map_dict(rgb2lab_map)))

In [ ]:
with gzip.open('/home/chitranshbose/Downloads/python_prac/django/img_proj/img_app/map.pkl.gz', 'rb') as f:
    color_map_complete = pk.load(f)
with gzip.open('/home/chitranshbose/Downloads/python_prac/django/img_proj/img_app/maprgb2lab.pkl.gz', 'rb') as f:
    maprgb2lab = pk.load(f)

In [ ]:
res = {}
for (rgb_key,lab_val), (lab_key,rgb_val) in zip(maprgb2lab.items(), color_map_complete.items()):
    res[rgb_key] = rgb_val

In [ ]:
with gzip.open('rgb_mapping.pkl.gz', 'wb') as f:
    pk.dump(res, f, protocol=pk.HIGHEST_PROTOCOL)

In [ ]:
# with open("/home/chitranshbose/Downloads/python_prac/django/img_proj/img_app/map.pkl", "rb") as file:
#     color_map_complete = pk.load(file)
#color_map_complete = joblib.load('/home/chitranshbose/Downloads/python_prac/django/img_proj/img_app/map.pkl')

# with open("/home/chitranshbose/Downloads/python_prac/django/img_proj/img_app/maprgb2lab.pkl", "rb") as file:
#     maprgb2lab=pk.load(file)
    
#maprgb2lab = joblib.load('/home/chitranshbose/Downloads/python_prac/django/img_proj/img_app/maprgb2lab.pkl')
# print("models loaded")

# with gzip.open('/home/chitranshbose/Downloads/python_prac/django/img_proj/img_app/map.pkl.gz', 'rb') as f:
#     color_map_complete = pk.load(f)

# with gzip.open('/home/chitranshbose/Downloads/python_prac/django/img_proj/img_app/maprgb2lab.pkl.gz', 'rb') as f:
#     maprgb2lab = pk.load(f)

In [ ]:
from PIL import Image
from time import time

In [ ]:
img_path = "/home/chitranshbose/Downloads/travel.jpg"
im = Image.open(img_path)
w, h = im.size
total = w*h
img_colors = list(im.getdata())


#img_colors_dict = Counter(img_colors)
                

In [ ]:
img_clr = im.getcolors(maxcolors=256*256*256)

In [ ]:
img_clr

In [ ]:
help(im.getcolors)

In [ ]:
dir(im.getcolors)

In [ ]:
img_path = "/home/chitranshbose/Downloads/travel.jpg"
im = Image.open(img_path)
w, h = im.size
total = w*h
st = time()
img_colors = list(im.getcolors(maxcolors=256*256*256))
print(time()-st)

In [ ]:
img_colors_dict = {x[1]:x[0] for x in img_colors}

In [ ]:
img_colors_dict

In [ ]:



img_colors
res_st = t.time()
# dt=np.dtype('int,int,int')
# img_colors = np.array(img_colors,dtype=dt)

img_colors_dict = Counter(img_colors)


# unique, counts = np.unique(img_colors, return_counts=True)
# print(f"unique: {unique}")
# img_colors_dict = dict(zip(unique, counts))
#print(f"img_colors: {type(img_colors[0])}\n")

res_en = t.time()
print(f"res time: {res_en-res_st}")
if img_obj.mode=='L':
    return render(req, 'show_color_shades.html',{'form':form,'msg':"The image is Grayscale."})

count_map = defaultdict(int)


for clr, cnt in img_colors_dict.items():
    count_map[rgb_mapping[clr[:3]]] += cnt
    
                    

#res = Counter({clr:round((count_map[clr]/total)*100,2) for clr,_ in count_map.items()})
res = {}
for clr,_ in count_map.items():
    res[clr] = round((count_map[clr]/total)*100,2)

res = Counter(res)


k = min(len(res), int(req.POST['drop_down']))
res = dict(res.most_common(k))